In [1]:
import numpy as np
import torch
import pandas as pd

import torch.nn as nn
from torch import optim
from torch.utils.data import TensorDataset, DataLoader
# from missforest import MissForest
from sklearn.ensemble import RandomForestRegressor
import warnings

In [2]:
n = 2*10**4
d = 10

lr = 0.01
l1_lambda = 0 #0.001
epochs = 100

def reg_func(x):
    y = 2 * (x[1] + 0.5)**(1/3) + np.exp((x[2] + x[3] + 1)/2) + 4 * x[3]**2    # Bayes risk approx 0.2290
    # y = 2 * (x[1] + 0.5)**(1/3) + np.exp((x[2] + x[3] + 1)/2) + (4 * x[3])**2    # Bayes risk approx 0.9025
    return y

In [3]:
X = np.random.uniform(-0.5 , 0.5, (n, d))
epsilon = np.random.normal(0, 0.25, n)
Y = np.zeros(n)
Y_true = np.zeros(n)
for i in range(n):
    Y_true[i] = reg_func(X[i,:])
    Y[i] = Y_true[i] + epsilon[i]

Omega = np.random.binomial(1, 0.5, (n, d))
sample_mean = np.sum(X*Omega, axis = 0) / np.sum(Omega, axis = 0)
Z_ZI = X * Omega

Z_ZI_train = Z_ZI[0:int(n/2), :]
Z_ZI_val = Z_ZI[int(n/2):int(3*n/4), :]
Z_ZI_test = Z_ZI[int(3*n/4):n, :]
Omega_train = Omega[0:int(n/2), :]
Omega_val = Omega[int(n/2):int(3*n/4), :]
Omega_test = Omega[int(3*n/4):n, :]

Y_train = Y[0:int(n/2)]
Y_val = Y[int(n/2):int(3*n/4)]
Y_test = Y[int(3*n/4):n]
Y_true_test = Y_true[int(3*n/4):n]

Z_ZI_train = torch.tensor(Z_ZI_train, dtype=torch.float32)
Z_ZI_val = torch.tensor(Z_ZI_val, dtype=torch.float32)
Z_ZI_test = torch.tensor(Z_ZI_test, dtype=torch.float32)
Omega_train = torch.tensor(Omega_train, dtype=torch.float32)
Omega_val = torch.tensor(Omega_val, dtype=torch.float32)
Omega_test = torch.tensor(Omega_test, dtype=torch.float32)

Y_train = torch.tensor(Y_train.reshape(-1, 1), dtype=torch.float32)
Y_val = torch.tensor(Y_val.reshape(-1, 1), dtype=torch.float32)
Y_test = torch.tensor(Y_test.reshape(-1, 1), dtype=torch.float32)
Y_true_test = torch.tensor(Y_true_test.reshape(-1, 1), dtype=torch.float32)

In [ ]:
### Pattern Embedding Neural Network (PENN)
class PENN(nn.Module):
    def __init__(self):
        super().__init__()
        f2_output_dim = 8
        embedding_dim = 2
        
        # Construct the neural network f2
        self.f2 = nn.Sequential(
            nn.Linear(d, 50),  
            nn.ReLU(),
            nn.Linear(50, 50),  
            nn.ReLU(),
            nn.Linear(50, 50),  
            nn.ReLU(),
            nn.Linear(50, f2_output_dim)
        )

        # Construct the neural network f3, i.e. the embedding function
        self.f3 = nn.Sequential(
            nn.Linear(d, 30),  
            nn.ReLU(),
            nn.Linear(30, 30),  
            nn.ReLU(),
            nn.Linear(30, 30),  
            nn.ReLU(),
            nn.Linear(30, embedding_dim)
        )

        
        # Construct the neural network f1
        self.f1 = nn.Sequential(
            nn.Linear(f2_output_dim + embedding_dim, 30),
            nn.ReLU(),
            nn.Linear(30, 30),
            nn.ReLU(),
            nn.Linear(30, 30),
            nn.ReLU(),
            nn.Linear(30, 1)  
        )
    
    # Combine f1, f2 and f3 to construct the Pattern Embedding Neural Network (PENN)
    def forward(self, z, omega):
        # compute the output of f2 and f3
        f2_output = self.f2(z)
        f3_output = self.f3(omega)
        
        # Concatenate the output of f2 and f3
        combined_features = torch.cat((f2_output, f3_output), dim=1)
        
        # Apply the combined network
        final_output = self.f1(combined_features)
        
        return final_output
    
model_PENN = PENN()
PENN_train_data = TensorDataset(Z_ZI_train, Omega_train, Y_train)
PENN_train_loader = DataLoader(dataset = PENN_train_data, batch_size=20, shuffle=True)

optimizer = optim.SGD(model_PENN.parameters(), lr=lr)
loss_fn = nn.MSELoss()

for epoch in range(epochs):

    for z_batch, omega_batch, y_batch in PENN_train_loader:
        optimizer.zero_grad()
        pred = model_PENN(z_batch, omega_batch)
        loss = loss_fn(pred, y_batch)

        # L1 penalty
        l1_penalty = 0
        for param in model_PENN.parameters():
            l1_penalty += torch.sum(torch.abs(param))
        # Add L1 penalty to the loss
        loss += l1_lambda * l1_penalty

        loss.backward()
        optimizer.step()

    model_PENN.eval()
    val_loss = loss_fn(model_PENN(Z_ZI_val, Omega_val), Y_val)

    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, train_loss: {loss.item()}, val_loss: {val_loss}')

print(f'test_loss: {loss_fn(model_PENN(Z_ZI_test, Omega_test), Y_test) - 0.2290}')

Epoch 10, train_loss: 0.10958679020404816, val_loss: 0.2893974483013153
Epoch 20, train_loss: 0.2710653245449066, val_loss: 0.28292107582092285
Epoch 30, train_loss: 0.21033737063407898, val_loss: 0.2724165916442871
Epoch 40, train_loss: 0.1927618831396103, val_loss: 0.2961089015007019
Epoch 50, train_loss: 0.17029690742492676, val_loss: 0.264447420835495
Epoch 60, train_loss: 0.1827601194381714, val_loss: 0.2651694416999817
Epoch 70, train_loss: 0.2090921401977539, val_loss: 0.2648721933364868
Epoch 80, train_loss: 0.31953901052474976, val_loss: 0.265537828207016
Epoch 90, train_loss: 0.21863345801830292, val_loss: 0.266249418258667
Epoch 100, train_loss: 0.14093215763568878, val_loss: 0.2672097086906433
test_loss: 0.25075796246528625


In [8]:
### Zero Imputation
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.nn = nn.Sequential(
            nn.Linear(d, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, 30),
            nn.ReLU(),
            nn.Linear(30, 30),
            nn.ReLU(),
            nn.Linear(30, 30),
            nn.ReLU(),
            nn.Linear(30, 1)  
        )
    
    def forward(self, x):
        final_output = self.nn(x)
        return final_output
    
model_NN = NN()
ZI_train_data = TensorDataset(Z_ZI_train, Y_train)
ZI_train_loader = DataLoader(dataset = ZI_train_data, batch_size=20, shuffle=True)

optimizer = optim.SGD(model_NN.parameters(), lr=lr)
loss_fn = nn.MSELoss()

for epoch in range(epochs):

    for z_batch, y_batch in ZI_train_loader:
        optimizer.zero_grad()
        pred = model_NN(z_batch)
        loss = loss_fn(pred, y_batch)

        # L1 penalty
        l1_penalty = 0
        for param in model_NN.parameters():
            l1_penalty += torch.sum(torch.abs(param))
        # Add L1 penalty to the loss
        loss += l1_lambda * l1_penalty

        loss.backward()
        optimizer.step()

    model_NN.eval()
    val_loss = loss_fn(model_NN(Z_ZI_val), Y_val)

    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, train_loss: {loss.item()}, val_loss: {val_loss}')

print(f'test_loss: {loss_fn(model_NN(Z_ZI_test), Y_test) - 0.2290}')

Epoch 10, train_loss: 0.41881242394447327, val_loss: 0.29276207089424133
Epoch 20, train_loss: 0.2133830338716507, val_loss: 0.28179872035980225
Epoch 30, train_loss: 0.31873372197151184, val_loss: 0.29176002740859985
Epoch 40, train_loss: 0.24851226806640625, val_loss: 0.29104724526405334
Epoch 50, train_loss: 0.26487264037132263, val_loss: 0.28024712204933167
Epoch 60, train_loss: 0.2127685844898224, val_loss: 0.2841445207595825
Epoch 70, train_loss: 0.2593303918838501, val_loss: 0.2784324288368225
Epoch 80, train_loss: 0.10762915760278702, val_loss: 0.2845163643360138
Epoch 90, train_loss: 0.17687493562698364, val_loss: 0.28605684638023376
Epoch 100, train_loss: 0.3109777271747589, val_loss: 0.28108516335487366
test_loss: 0.04041251540184021
